In [ ]:
import os
print(os.getcwd())

In [25]:
from langchain_openai import ChatOpenAI
import httpx

# Create an HTTP client that skips SSL verification (only for hackathon/test environments)
client = httpx.Client(verify=False)
llm = ChatOpenAI(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-gpt-4o",
 api_key="sk-u6zTQaiDKlhHn4-k_hhihw",
 http_client=client
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-text-embedding-3-large",
 api_key="sk-u6zTQaiDKlhHn4-k_hhihw",
 http_client=client)

import requests
for method in ("get","post","put","delete","head","options","patch"):
    original = getattr(requests,method)

    def insecure_request(*args, _original = original, **kwargs):
        kwargs["verify"] = False
        return _original(*args,**kwargs)
    
    setattr(requests,method,insecure_request)

In [30]:
import os
import json
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

# -------------------------------
# 1. Loader: read all .txt files
# -------------------------------
def load_txts():
    txt_folder = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\txt_files"
    txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.lower().endswith(".txt")]

    all_docs = []
    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as f:
            content = f.read()
            # Return LangChain Document objects
            doc = Document(
                page_content=content,
                metadata={"source": os.path.basename(txt_file)}
            )
            all_docs.append(doc)

    return all_docs

# -------------------------------
# 2. Loader: read JSON EHR file
# -------------------------------
def load_jsons():
    file_path = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\json_files\mock_ehr.json"
    with open(file_path, "r", encoding="utf-8") as f:
        ehr_Report = json.load(f)
    return ehr_Report

def get_patient_by_id(patient_id):
    ehr_data = load_jsons()
    for patient in ehr_data.get("patients", []):
        if patient.get("id") == patient_id:
            return patient
    return None 

patient_record_json = get_patient_by_id("P001")

# -------------------------------
# 3. Helper: convert JSON to text
# -------------------------------
def json_to_text(d, prefix="- "):
    lines = []
    for key, value in d.items():
        key_name = key.replace("_", " ").title()
        if isinstance(value, dict):
            lines.append(f"{prefix}{key_name}:")
            lines.extend([f"  {line}" for line in json_to_text(value, prefix=prefix)])
        elif isinstance(value, list):
            if all(isinstance(i, dict) for i in value):
                lines.append(f"{prefix}{key_name}:")
                for item in value:
                    lines.extend([f"  {line}" for line in json_to_text(item, prefix=prefix)])
            else:
                lines.append(f"{prefix}{key_name}: {', '.join(map(str,value))}")
        else:
            lines.append(f"{prefix}{key_name}: {value}")
    return lines

# -------------------------------
# 4. Retriever builder
# -------------------------------
def getRetriever():
    txt_documents = load_txts()
    chunked_docs = []
    for doc in txt_documents:
        # Prompt template for chunking
        chunk_prompt = ChatPromptTemplate.from_messages([
            ("system", 
                "You are a helpful assistant that splits medical or disease-related documents "
                "into semantically meaningful chunks for downstream semantic search and retrieval."
            ),
            ("human", 
                """You will be given a document. Split it into semantically coherent sections, following these rules:

                1. Do not omit any information.
                2. Keep related items together.
                3. Each chunk should be self-contained.
                4. Aim for ~5000–6000 words per chunk more or less.
                5. Return chunks as a numbered list with headings like '### Chunk 1:', '### Chunk 2:'.
                6. Include metadata (e.g., source) at the beginning of each chunk.

                Document:
                {document}
                """
            )
        ])

        formatted_prompt = chunk_prompt.format_messages(document=doc.page_content)
        response = llm.invoke(formatted_prompt)

        # Split on "### Chunk" headings
        chunks = [c.strip() for c in response.content.split("### Chunk") if c.strip()]

        for idx, chunk in enumerate(chunks):
            chunked_docs.append(
                Document(
                    page_content=chunk,
                    metadata={"source": f"{doc.metadata['source']}_chunk{idx+1}"}
                )
            )

    # Build vector store
    vector_store = Chroma.from_documents(
        chunked_docs,
        embedding_model,
        persist_directory="chroma_db"
    )

    # Prepare patient record text
    patient_record = "Patient Past Record:\n" + "\n".join(json_to_text(patient_record_json)) \
        if patient_record_json else "No past record found."

    retriever = vector_store.as_retriever(search_kwargs={"k": 3})

    # Prompt for answering queries
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system",
            "You are a helpful medical assistant. Answer questions using only the provided context. "
            "If the answer is not in the context, say you don't know rather than making up information."
        ),
        ("human",
            """You are given the patient's past medical records and the retrieved context from relevant documents.

            Context (retrieved from vector database):
            {context}

            Patient Past Record:
            {patient_record}

            Current patient query / symptom:
            {question}

            Guidelines:
            1. Use ONLY the information from the retrieved context and patient past record.
            2. Synthesize both sources to provide a clear, concise response.
            3. Do NOT invent facts not present in the context or past record.
            4. Provide your response in the following structured format:

                - **Triage Level**: [Low / Medium / High / Emergency]
                - **Reasoning**: [Explain why this triage level is assigned]
                - **Urgent Evaluation Needed**: [Yes/No; specify tests if any]
                - **Patient Actions**: [What patient should do next]
                - **Clinician Tasks**: [Tasks clinician should perform]
                - **Disclaimer**: [Include standard medical disclaimer]

            5. If the context and past record do not contain the answer, respond:
                "The provided documents do not contain information about this."
            """
        )
    ])

    # Build RAG chain with patient_record injected
    rag_chain = (
        {
            "context": retriever,
            "patient_record": lambda _: patient_record,
            "question": RunnablePassthrough()
        }
        | qa_prompt
        | llm
    )
    return rag_chain

# -------------------------------
# 5. Run a query
# -------------------------------
if __name__ == "__main__":
    query = "I'm very thirsty and peeing a lot, even at night."
    rag_chain = getRetriever()
    response = rag_chain.invoke(query)
    print(response.content)


- **Triage Level**: Medium  
- **Reasoning**: The symptoms of increased thirst and urination could indicate poor blood sugar control, which is concerning given the patient's history of Type 2 Diabetes. These symptoms may suggest hyperglycemia (high blood sugar levels) or potential complications such as undiagnosed diabetes progression, which requires prompt medical evaluation.  
- **Urgent Evaluation Needed**: Yes; the patient should have their blood sugar levels checked immediately through fasting glucose, hemoglobin A1c, or random glucose testing. Urinalysis to check for glucose or ketones in the urine may also be considered.  
- **Patient Actions**:  
  1. Contact your primary care provider immediately to report symptoms.  
  2. Monitor blood sugar levels if you have a glucometer and share results with your clinician.  
  3. Maintain proper hydration and avoid sugary beverages until seen by a healthcare provider.  
  4. Do not delay seeking care, especially if symptoms worsen or you

In [ ]:
# def load_txts():
#     txt_folder = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\txt_files"
#     txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.lower().endswith(".txt")]

#     all_docs = []

#     for txt_file in txt_files:
#         with open(txt_file, "r", encoding="utf-8") as f:
#             content = f.read()

#             # Create a simple document-like dict (similar to PDF loader output)
#             doc = {
#                 "page_content": content,
#                 "metadata": {"source": os.path.basename(txt_file)}
#             }
#             all_docs.append(doc)

#      # Combine all text into one big string
#     full_text = "\n".join([doc["page_content"] for doc in all_docs])
#     return full_text
#     # return all_docs

In [ ]:
# import os
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.messages import HumanMessage
# from langchain_core.documents import Document
# from langchain_community.vectorstores import Chroma
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.runnables import RunnableMap
# from loader import load_txts
# from azure_llm import getMassGpt
# from embedding import getLargeEmbedding

# def getRetriever():
#     txt_documents = load_txts()

#     # combined_texts = [
#     #     {"text": full_text1, "source_id": "PDF_Doc1"},
#     #     {"text": full_text2, "source_id": "TXT_Doc2"}
#     # ]

#     llm = getMassGpt()
#     embedding_model = getLargeEmbedding()

#     chunked_docs = []
#     for doc in txt_documents:
#         # Use your existing chunking prompt
#         prompt = ChatPromptTemplate.from_messages([
#             ("system", 
#                 "You are a helpful assistant that splits medical or disease-related documents "
#                 "into semantically meaningful chunks for downstream semantic search and retrieval."
#             ),
#             ("human", 
#                 """You will be given a document. Split it into semantically coherent sections, following these rules:

#                 1. **Completeness**: Do not omit any information — every detail must appear in some chunk.
#                 2. **Semantic grouping**: Keep related items (like lists of symptoms, causes, treatments, dates, or tables) together in the same chunk.
#                 3. **Self-contained**: Each chunk should be understandable on its own without needing other chunks.
#                 4. **Chunk size**: Aim for ~5000–6000 words per chunk. If the document is shorter, keep it intact. 
#                     If longer, split carefully without breaking sentences, lists, or tables.
#                 5. **Formatting**: Return the chunks as a numbered list. Clearly separate each chunk with a heading like 'Chunk 1:', 'Chunk 2:', etc.
#                 6. **Metadata preservation**: If the document contains metadata (e.g., disease name, category, source), 
#                     include it at the beginning of the relevant chunk.

#                 Document:
#                 {document}
#                 """
#             )
#         ])

#         formatted_prompt = prompt.format_messages(document=doc['page_content'])
#         response = llm.invoke(formatted_prompt)
#         chunks = response.content.split("\n\n")
        
#         for idx, chunk in enumerate(chunks):
#             if chunk.strip():
#                 chunked_docs.append(
#                     Document(
#                         page_content=chunk.strip(),
#                         metadata={"source_id": f"{doc['source_id']}_chunk{idx+1}"}
#                     )
#                 )

#     vector_store = Chroma.from_documents(
#         chunked_docs,
#         embedding_model,
#         persist_directory="chroma_db"
#     )

#     retriever = vector_store.as_retriever(search_kwargs={"k": 3})
#     rag_chain = (
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#     )
#     return rag_chain

In [ ]:
# query = "Forecast the next 12 months of sales for Laptops"
# rag_chain = getRetriever()
# response = rag_chain.invoke(query)
# print(response.content)

In [ ]:
# def json_to_text(d, prefix="- "):
#     lines = []
#     for key, value in d.items():
#         key_name = key.replace("_", " ").title()
#         if isinstance(value, dict):
#             lines.append(f"{prefix}{key_name}:")
#             lines.extend([f"  {line}" for line in json_to_text(value, prefix=prefix)])
#         elif isinstance(value, list):
#             if all(isinstance(i, dict) for i in value):
#                 lines.append(f"{prefix}{key_name}:")
#                 for item in value:
#                     lines.extend([f"  {line}" for line in json_to_text(item, prefix=prefix)])
#             else:
#                 lines.append(f"{prefix}{key_name}: {', '.join(map(str,value))}")
#         else:
#             lines.append(f"{prefix}{key_name}: {value}")
#     return lines

# formatted_record = "Patient Past Record:\n" + "\n".join(json_to_text(patient_record))
# print(formatted_record)


In [ ]:
# import os
# import json
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.documents import Document
# from langchain_community.vectorstores import Chroma
# from langchain_core.runnables import RunnablePassthrough

# # -------------------------------
# # 1. Loader: read all .txt files
# # -------------------------------
# def load_txts():
#     txt_folder = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\txt_files"
#     txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.lower().endswith(".txt")]

#     all_docs = []
#     for txt_file in txt_files:
#         with open(txt_file, "r", encoding="utf-8") as f:
#             content = f.read()
#             # Return LangChain Document objects
#             doc = Document(
#                 page_content=content,
#                 metadata={"source": os.path.basename(txt_file)}
#             )
#             all_docs.append(doc)

#     return all_docs

# def load_jsons():
#     file_path = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\json_files\mock_ehr.json"
#     # Open and load JSON
#     with open(file_path, "r", encoding="utf-8") as f:
#         ehr_Report = json.load(f)
#     return ehr_Report

# def get_patient_by_id(patient_id):
#     ehr_data = load_jsons()
#     for patient in ehr_data.get("patients", []):
#         if patient.get("id") == patient_id:
#             return patient
#     return None 


# patient_record_json = get_patient_by_id("P001")

# def json_to_text(d, prefix="- "):
#     lines = []
#     for key, value in d.items():
#         key_name = key.replace("_", " ").title()
#         if isinstance(value, dict):
#             lines.append(f"{prefix}{key_name}:")
#             lines.extend([f"  {line}" for line in json_to_text(value, prefix=prefix)])
#         elif isinstance(value, list):
#             if all(isinstance(i, dict) for i in value):
#                 lines.append(f"{prefix}{key_name}:")
#                 for item in value:
#                     lines.extend([f"  {line}" for line in json_to_text(item, prefix=prefix)])
#             else:
#                 lines.append(f"{prefix}{key_name}: {', '.join(map(str,value))}")
#         else:
#             lines.append(f"{prefix}{key_name}: {value}")
#     return lines

# # -------------------------------
# # 2. Retriever builder
# # -------------------------------
# def getRetriever():
#     txt_documents = load_txts()
#     chunked_docs = []

#     for doc in txt_documents:
#         # Prompt template
#         prompt = ChatPromptTemplate.from_messages([
#             ("system", 
#                 "You are a helpful assistant that splits medical or disease-related documents "
#                 "into semantically meaningful chunks for downstream semantic search and retrieval."
#             ),
#             ("human", 
#                 """You will be given a document. Split it into semantically coherent sections, following these rules:

#                 1. Do not omit any information.
#                 2. Keep related items together.
#                 3. Each chunk should be self-contained.
#                 4. Aim for ~5000–6000 words per chunk more or less.
#                 5. Return chunks as a numbered list with headings like 'Chunk 1:', 'Chunk 2:'.
#                 6. Include metadata (e.g., source) at the beginning of each chunk.

#                 Document:
#                 {document}
#                 """
#             )
#         ])

#         # Format prompt with actual document text
#         formatted_prompt = prompt.format_messages(document=doc.page_content)
#         response = llm.invoke(formatted_prompt)

#         # Split on "Chunk" headings instead of fragile \n\n
#         chunks = [c.strip() for c in response.content.split("Chunk") if c.strip()]

#         for idx, chunk in enumerate(chunks):
#             chunked_docs.append(
#                 Document(
#                     page_content=chunk,
#                     metadata={"source": f"{doc.metadata['source']}_chunk{idx+1}"}
#                 )
#             )

#     # Build vector store
#     vector_store = Chroma.from_documents(
#         chunked_docs,
#         embedding_model,
#         persist_directory="chroma_db"
#     )

#     patient_record = "Patient Past Record:\n" + "\n".join(json_to_text(patient_record_json))
#     retriever = vector_store.as_retriever(search_kwargs={"k": 3})

#     prompt = ChatPromptTemplate.from_messages([
#         ("system",
#             "You are a helpful medical assistant. Answer questions using only the provided context. "
#             "If the answer is not in the context, say you don't know rather than making up information."
#         ),
#         ("human",
#             """You are given the patient's past medical records and the retrieved context from relevant documents.

#             Context (retrieved from vector database):
#             {context}

#             Patient Past Record:
#             {patient_record}

#             Current patient query / symptom:
#             {question}

#             Guidelines:
#             1. Use ONLY the information from the retrieved context and patient past record.
#             2. Synthesize both sources to provide a clear, concise response.
#             3. Do NOT invent facts not present in the context or past record.
#             4. Provide your response in the following structured format:

#                 - **Triage Level**: [Low / Medium / High / Emergency]
#                 - **Reasoning**: [Explain why this triage level is assigned]
#                 - **Urgent Evaluation Needed**: [Yes/No; specify tests if any]
#                 - **Patient Actions**: [What patient should do next]
#                 - **Clinician Tasks**: [Tasks clinician should perform]
#                 - **Disclaimer**: [Include standard medical disclaimer]

#             5. If the context and past record do not contain the answer, respond:
#                 "The provided documents do not contain information about this."
#             """
#         )
#     ])


#     # Build RAG chain
#     rag_chain = (
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#     )
#     return rag_chain



In [28]:
query = "I'm very thirsty and peeing a lot, even at night."
rag_chain = getRetriever()
response = rag_chain.invoke(query)
print(response.content)

KeyError: "Input to ChatPromptTemplate is missing variables {'patient_record'}.  Expected: ['context', 'patient_record', 'question'] Received: ['context', 'question']\nNote: if you intended {patient_record} to be part of the string and not a variable, please escape it with double curly braces like: '{{patient_record}}'.\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/INVALID_PROMPT_INPUT "

In [ ]:
# import os
# import json
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.documents import Document
# from langchain_community.vectorstores import Chroma
# from langchain_core.runnables import RunnablePassthrough

# # -------------------------------
# # 1. Loader: read all .txt files
# # -------------------------------
# def load_txts():
#     txt_folder = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\txt_files"
#     txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.lower().endswith(".txt")]

#     all_docs = []
#     for txt_file in txt_files:
#         with open(txt_file, "r", encoding="utf-8") as f:
#             content = f.read()
#             # Return LangChain Document objects
#             doc = Document(
#                 page_content=content,
#                 metadata={"source": os.path.basename(txt_file)}
#             )
#             all_docs.append(doc)

#     return all_docs

# def load_jsons():
#     file_path = r"C:\Users\GenAIBLRANCUSR33\Desktop\Team7\database\json_files\mock_ehr.json"
#     # Open and load JSON
#     with open(file_path, "r", encoding="utf-8") as f:
#         ehr_Report = json.load(f)
#     return ehr_Report

# def get_patient_by_id(patient_id):
#     ehr_data = load_jsons()
#     for patient in ehr_data.get("patients", []):
#         if patient.get("id") == patient_id:
#             return patient
#     return None 


# patient_record_json = get_patient_by_id("P001")

# def json_to_text(d, prefix="- "):
#     lines = []
#     for key, value in d.items():
#         key_name = key.replace("_", " ").title()
#         if isinstance(value, dict):
#             lines.append(f"{prefix}{key_name}:")
#             lines.extend([f"  {line}" for line in json_to_text(value, prefix=prefix)])
#         elif isinstance(value, list):
#             if all(isinstance(i, dict) for i in value):
#                 lines.append(f"{prefix}{key_name}:")
#                 for item in value:
#                     lines.extend([f"  {line}" for line in json_to_text(item, prefix=prefix)])
#             else:
#                 lines.append(f"{prefix}{key_name}: {', '.join(map(str,value))}")
#         else:
#             lines.append(f"{prefix}{key_name}: {value}")
#     return lines

# patient_record = "Patient Past Record:\n" + "\n".join(json_to_text(patient_record_json))

# # -------------------------------
# # 2. Retriever builder
# # -------------------------------
# def getRetriever():
#     txt_documents = load_txts()
#     chunked_docs = []

#     for doc in txt_documents:
#         # Escape curly braces in document content
#         safe_doc_content = doc.page_content.replace("{", "{{").replace("}", "}}")

#         # Prompt template
#         prompt = ChatPromptTemplate.from_messages([
#             ("system",
#              "You are a helpful assistant that splits medical or disease-related documents "
#              "into semantically meaningful chunks for downstream semantic search and retrieval."),
#             ("human",
#              """You will be given a document. Split it into semantically coherent sections.

#              Guidelines:
#              1. Do not omit any information.
#              2. Keep related items together.
#              3. Each chunk should be self-contained.
#              4. Aim for ~800–1000 words per chunk.
#              5. Include metadata (e.g., source) at the beginning of each chunk.
#              6. Return output in strict JSON format as a list:
#                 [{"chunk": "text of chunk 1", "source": "source_name"}, ...]

#              Document:
#              {document}
#              """)
#         ])

#         # Correct formatting with escaped document
#         formatted_prompt = prompt.format_messages(document=safe_doc_content)

#         # Call your LLM
#         response = llm.invoke(formatted_prompt)

#         # Parse JSON safely
#         try:
#             chunks_json = json.loads(response.content)
#             if not isinstance(chunks_json, list):
#                 raise ValueError("LLM output is not a list")
#         except Exception as e:
#             print(f"Warning: JSON parse failed for document {doc.metadata.get('source', 'unknown')}. Using fallback. Error: {e}")
#             chunks_json = [{"chunk": p.strip(), "source": doc.metadata.get("source", "unknown")}
#                            for p in doc.page_content.split("\n\n") if p.strip()]

#         # Add chunks to list
#         for idx, chunk_data in enumerate(chunks_json):
#             if isinstance(chunk_data, dict):
#                 page_content = chunk_data.get("chunk", "")
#                 source = chunk_data.get("source", f"{doc.metadata.get('source', 'unknown')}_chunk{idx+1}")
#             else:
#                 page_content = str(chunk_data)
#                 source = f"{doc.metadata.get('source', 'unknown')}_chunk{idx+1}"

#             chunked_docs.append(
#                 Document(
#                     page_content=page_content,
#                     metadata={"source": source, "chunk_idx": idx+1}
#                 )
#             )

#     # Build vector store
#     vector_store = Chroma.from_documents(
#         chunked_docs,
#         embedding_model,  # Ensure embedding_model is defined
#         persist_directory="chroma_db"
#     )

#     # Return retriever for top-k chunks
#     retriever = vector_store.as_retriever(search_kwargs={"k": 3})



#     prompt = ChatPromptTemplate.from_messages([
#         ("system",
#             "You are a helpful medical assistant. Answer questions using only the provided context. "
#             "If the answer is not in the context, say you don't know rather than making up information."
#         ),
#         ("human",
#             """You are given the patient's past medical records and the retrieved context from relevant documents.

#             Context (retrieved from vector database):
#             {context}

#             Patient Past Record:
#             {patient_record}

#             Current patient query / symptom:
#             {question}

#             Guidelines:
#             1. Use ONLY the information from the retrieved context and patient past record.
#             2. Synthesize both sources to provide a clear, concise response.
#             3. Do NOT invent facts not present in the context or past record.
#             4. Provide your response in the following structured format:

#                 - **Triage Level**: [Low / Medium / High / Emergency]
#                 - **Reasoning**: [Explain why this triage level is assigned]
#                 - **Urgent Evaluation Needed**: [Yes/No; specify tests if any]
#                 - **Patient Actions**: [What patient should do next]
#                 - **Clinician Tasks**: [Tasks clinician should perform]
#                 - **Disclaimer**: [Include standard medical disclaimer]

#             5. If the context and past record do not contain the answer, respond:
#                 "The provided documents do not contain information about this."
#             """
#         )
#     ])

#     # Build RAG chain
#     rag_chain = (
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#     )
#     return rag_chain


In [ ]:
query = "I'm very thirsty and peeing a lot, even at night."
rag_chain = getRetriever()
response = rag_chain.invoke(query)
print(response.content)

In [ ]:
from common_util import find_matching_slots,book_slot,suggest_alternatives_json

def doctor_appoint_agent(state: dict) -> dict:
    print("Entered doctor_appoint_agent")
    preferred, alternatives = find_matching_slots("doctor", state["doctor_name"], state["preferred_start"])
    if preferred and book_slot("doctor", state["doctor_name"], state["preferred_start"]):
        state["status"] = "confirmed"
        state["booked_slot"] = state["preferred_start"]
        state["output"] = f"Doctor appointment with {state['doctor_name']} confirmed for {state['preferred_start']}."
    else:
        state["status"] = "unavailable"
        state["alternatives"] = suggest_alternatives_json(alternatives)
        state["output"] = f"Requested slot unavailable. Alternatives provided."
    return state

from common_util import find_matching_slots,book_slot,suggest_alternatives_json

def disease_appoint_agent(state: dict) -> dict:
    preferred, alternatives = find_matching_slots("disease", state["disease"], state["preferred_start"])
    if preferred and book_slot("disease", state["disease"], state["preferred_start"]):
        state["status"] = "confirmed"
        state["booked_slot"] = state["preferred_start"]
        state["output"] = f"Consultation for {state['disease']} with {state['specialty']} confirmed for {state['preferred_start']}."
    else:
        state["status"] = "unavailable"
        state["alternatives"] = suggest_alternatives_json(alternatives)
        state["output"] = f"Requested consultation slot unavailable. Alternatives provided."
    return state

def fallback_appoint_agent(state: dict) -> dict:
    state["status"] = "fallback"
    state["output"] = "Sorry, I couldn’t classify your request into doctor, lab, disease, or service."
    return state

from common_util import find_matching_slots,book_slot,suggest_alternatives_json

def lab_appoint_agent(state: dict) -> dict:
    preferred, alternatives = find_matching_slots("lab", state["test"], state["preferred_start"])
    if preferred and book_slot("lab", state["test"], state["preferred_start"]):
        state["status"] = "confirmed"
        state["booked_slot"] = state["preferred_start"]
        state["output"] = f"Lab test '{state['test']}' booked for {state['preferred_start']} at {state.get('location','default lab')}."
    else:
        state["status"] = "unavailable"
        state["alternatives"] = suggest_alternatives_json(alternatives)
        state["output"] = f"Requested lab slot unavailable. Alternatives provided."
    return state

from common_util import find_matching_slots,book_slot,suggest_alternatives_json

def service_appoint_agent(state: dict) -> dict:
    preferred, alternatives = find_matching_slots("service", state["service"], state["preferred_start"])
    if preferred and book_slot("service", state["service"], state["preferred_start"]):
        state["status"] = "confirmed"
        state["booked_slot"] = state["preferred_start"]
        state["output"] = f"Service '{state['service']}' booked for {state['preferred_start']}."
    else:
        state["status"] = "unavailable"
        state["alternatives"] = suggest_alternatives_json(alternatives)
        state["output"] = f"Requested service slot unavailable. Alternatives provided."
    return state

# import sqlite3, json

# def find_matching_slots(resource_type, resource_id, preferred_start):
#     conn = sqlite3.connect("triage.db")
#     conn.row_factory = sqlite3.Row
#     c = conn.cursor()

#     # Check if preferred slot exists
#     c.execute("""
#         SELECT * FROM availability
#         WHERE resource_type=? AND resource_id=? AND slot_start=?
#     """, (resource_type, resource_id, preferred_start))
#     preferred = c.fetchone()

#     # Get up to 5 alternative slots
#     c.execute("""
#         SELECT * FROM availability
#         WHERE resource_type=? AND resource_id=? AND is_available=1
#         ORDER BY slot_start ASC
#         LIMIT 5
#     """, (resource_type, resource_id))
#     alternatives = c.fetchall()

#     conn.close()
#     return preferred, alternatives

# def suggest_alternatives_json(alts):
#     return json.dumps([dict(r) for r in alts], indent=2)

# def book_slot(resource_type, resource_id, slot_start):
#     conn = sqlite3.connect("triage.db")
#     c = conn.cursor()
#     c.execute("""
#         UPDATE availability
#         SET is_available=0
#         WHERE resource_type=? AND resource_id=? AND slot_start=? AND is_available=1
#     """, (resource_type, resource_id, slot_start))
#     ok = (c.rowcount == 1)
#     conn.commit()
#     conn.close()
#     return ok

import sqlite3
import json

DB_FILE = "triage.db"

def find_matching_slots(resource_type, resource_id, preferred_start):
    """Find the preferred slot and up to 5 alternative available slots."""
    with sqlite3.connect(DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        c = conn.cursor()

        # Preferred slot
        c.execute(
            """
            SELECT * FROM availability
            WHERE resource_type=? AND resource_id=? AND slot_start=?
            """,
            (resource_type, resource_id, preferred_start),
        )
        preferred = c.fetchone()

        # Alternatives
        c.execute(
            """
            SELECT * FROM availability
            WHERE resource_type=? AND resource_id=? AND is_available=1
            ORDER BY slot_start ASC
            LIMIT 5
            """,
            (resource_type, resource_id),
        )
        alternatives = c.fetchall()

    return preferred, alternatives


def suggest_alternatives_json(alts):
    """Convert alternative slots into JSON format."""
    return json.dumps([dict(r) for r in alts], indent=2)


def book_slot(resource_type, resource_id, slot_start):
    """Book a slot by marking it unavailable. Returns True if successful."""
    with sqlite3.connect(DB_FILE) as conn:
        c = conn.cursor()
        c.execute(
            """
            UPDATE availability
            SET is_available=0
            WHERE resource_type=? AND resource_id=? AND slot_start=? AND is_available=1
            """,
            (resource_type, resource_id, slot_start),
        )
        ok = (c.rowcount == 1)
        conn.commit()

    return ok

